# Synapse Labeling Orphans

In [1]:
import datajoint as dj
import numpy as np
from scipy.spatial import KDTree
import time
from tqdm import tqdm
import ipyvolume.pylab as p3


In [21]:
dj.config['display.limit'] = 20
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

In [3]:
pinky.OverlayedSpineLabel()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,vertices,triangles
3,648518346341371119,0.35,=BLOB=,=BLOB=
3,648518346349386137,0.35,=BLOB=,=BLOB=
3,648518346349470171,0.35,=BLOB=,=BLOB=
3,648518346349471156,0.35,=BLOB=,=BLOB=
3,648518346349471500,0.35,=BLOB=,=BLOB=
3,648518346349471562,0.35,=BLOB=,=BLOB=
3,648518346349471565,0.35,=BLOB=,=BLOB=
3,648518346349471910,0.35,=BLOB=,=BLOB=
3,648518346349472574,0.35,=BLOB=,=BLOB=
3,648518346349472601,0.35,=BLOB=,=BLOB=


In [4]:
schema = dj.schema('microns_pinky')

@schema
class SynapseCompartmentLabelFinalThresholdRevised(dj.Manual):
    definition = """
    -> pinky.Synapse
    ---
    (postsynaptic_label)->pinky.LabelKey
    """ # (postsynaptic_label)->pinky.LabelKey : tinyint unsigned

In [5]:
segment_ids = (pinky.CoarseLabelOrphan - pinky.SegmentExclude).fetch('segment_id')
mesh_fetch = (pinky.Decimation35OrphanStitched & pinky.CompartmentOrphan.proj()).fetch()
label_fetch = (pinky.CoarseLabelOrphan - pinky.SegmentExclude).fetch()

meshes = dict()
labels = dict()
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6])
    
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7])
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

100%|██████████| 840/840 [01:04<00:00, 12.60it/s]


In [6]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    """ Old way of inserting without length threshold
    for synapse_id, nearest_node in zip(synapse_ids, nearest_nodes):
        synapse_labels[synapse_id] = vertex_labels[nearest_node]"""
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/840 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
184it [00:00, 163472.13it/s]
0it [00:00, ?it/s]
  0%|          | 3/840 [00:00<00:52, 15.92it/s]
0it [00:00, ?it/s]
  0%|          | 4/840 [00:00<01:05, 12.73it/s]
0it [00:00, ?it/s]
11it [00:00, 16465.86it/s]
0it [00:00, ?it/s]
178it [00:00, 160074.21it/s]
0it [00:00, ?it/s]
  1%|          | 7/840 [00:00<01:01, 13.50it/s]
0it [00:00, ?it/s]
  1%|          | 8/840 [00:00<01:34,  8.80it/s]
0it [00:00, ?it/s]
61it [00:00, 63220.30it/s]
0it [00:00, ?it/s]
  1%|          | 10/840 [00:00<01:34,  8.75it/s]
0it [00:00, ?it/s]
92it [00:00, 112928.29it/s]
0it [00:00, ?it/s]
  1%|▏         | 12/840 [00:01<02:34,  5.35it/s]
0it [00:00, ?it/s]
215it [00:00, 184487.59it/s]
0it [00:00, ?it/s]
  2%|▏         | 14/840 [00:01<02:18,  5.97it/s]
0it [00:00, ?it/s]
150it [00:00, 127383.19it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  2%|▏         | 17/840 [00:01<01:44,  7.85it/s]
0it [00:00, ?it/s]
1it [00:00, 1504.95it/s]
0it [00

In [ ]:
for syn in synapses_to_exclude:
    syn["criteria_id"] = 0

In [7]:
print(len(synapses_to_exclude))

0


In [8]:
pinky.SynapseExclude.insert(synapses_to_exclude,skip_duplicates=True)

In [9]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 840/840 [00:00<00:00, 6084.54it/s]


24.584077835083008


# Spine Synapse Labeler

In [31]:
pinky.SynapseSpineLabelFinalThresholdRevised() & "spine_label=10"

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment


In [10]:
@schema
class SynapseSpineLabelFinalThresholdRevised(dj.Manual):
    definition = """
    # Denotes which synapses are also on Spine Heads, Spines, or Spine Necks
    -> pinky.SynapseCompartmentLabelFinalThresholdRevised
    ---
    (spine_label)->pinky.LabelKey
    """ # (spine_label)->pinky.LabelKey : tinyint unsigned

In [11]:
mesh_fetch = np.array((pinky.Decimation35OrphanStitched & pinky.CompartmentOrphan.proj()).fetch('segment_id', 'vertices', 'triangles')).T
label_fetch = np.array(pinky.OverlayedSpineLabelOrphan.fetch('segment_id', 'vertices', 'triangles')).T

meshes = dict()
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch):
    meshes[segment_id] = (mesh_vertices, mesh_triangles)

labels = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch):
    labels[segment_id] = (label_vertices, label_triangles)
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x','postsyn_y', 'postsyn_z')).T

100%|██████████| 840/840 [01:02<00:00, 13.07it/s]


In [12]:
matched_synapses = dict()
synapses_to_exclude_spine = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    """ OLD WAY WITHOUTH LENGTH THRESHOLD
    for synapse_id, nearest_node in zip(synapse_ids, nearest_nodes):
        synapse_labels[synapse_id] = vertex_labels[nearest_node]
    """
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude_spine.append(dict(segmentation=3,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/840 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 2/840 [00:00<00:44, 18.74it/s]
0it [00:00, ?it/s]
204it [00:00, 207176.28it/s]
0it [00:00, ?it/s]
  0%|          | 4/840 [00:00<00:56, 14.84it/s]
0it [00:00, ?it/s]
11it [00:00, 18732.17it/s]
0it [00:00, ?it/s]
178it [00:00, 185625.59it/s]
0it [00:00, ?it/s]
  1%|          | 7/840 [00:00<00:55, 15.12it/s]
0it [00:00, ?it/s]
  1%|          | 8/840 [00:00<01:30,  9.20it/s]
0it [00:00, ?it/s]
61it [00:00, 73038.12it/s]
0it [00:00, ?it/s]
  1%|          | 10/840 [00:00<01:29,  9.30it/s]
0it [00:00, ?it/s]
92it [00:00, 115015.19it/s]
0it [00:00, ?it/s]
  1%|▏         | 12/840 [00:01<02:31,  5.47it/s]
0it [00:00, ?it/s]
215it [00:00, 219891.58it/s]
0it [00:00, ?it/s]
  2%|▏         | 14/840 [00:01<02:15,  6.11it/s]
0it [00:00, ?it/s]
150it [00:00, 194963.00it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
67it [00:00, 78650.54it/s]
0it [00:00, ?it/s]
1it [00:00, 1281.88it/s]
0it [00:00, ?it/s]
  2%|▏    

In [13]:
#####STILL NEED TO WRITE THE SYNAPSES TO EXCLUDE TO DATABASE

In [14]:
print(len(synapses_to_exclude_spine))


0


In [ ]:
for syn in synapses_to_exclude_spine:
    syn["criteria_id"] = 0

In [20]:
synapses_to_exclude_spine

[]

In [15]:
#add to synapse exclude
pinky.SynapseExclude.insert(synapses_to_exclude_spine,skip_duplicates=True)

In [16]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>15] = 1
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]<=15]
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]>12]

start = time.time()
SynapseSpineLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 840/840 [00:00<00:00, 7945.75it/s]


21.57568335533142


## Visualization of Labeled Synapses (no labeled Spines yet)

In [17]:
# pinky.LabelKey()

In [18]:
# pinky.ProofreadLabel & 'segment_id=648518346341393609'

In [19]:
# label_keys = [( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
#                ( 2, 'Apical', 'ignore blue'), ( 3, 'Basal', 'ignore yellow'),
#                ( 4, 'Oblique', 'ignore green'), ( 5, 'Soma', 'ignore red'),
#                ( 6, 'Axon-Soma', 'ignore aqua'), ( 7, 'Axon-Dendr', 'ignore off blue'),
#                ( 8, 'Dendrite', 'ignore purple'), ( 9, 'Distal', 'ignore pink'),
#                (10, 'ignore Error', 'brown'), (11, 'Unlabelable', 'ignore tan'),
#                (12, 'Cilia', 'ignore orange'), (13, 'Spine Head (unsure if correct label)', 'red'),
#                (14, 'Spine (unsure if correct label)', 'blue'),
#                (15, 'Spine Neck (unsure if correct label)', 'ignore green'),
#                (16, 'Bouton (unsure if correct label)', 'ignore')]
# segment_id = 648518346341393609
# synapse_label_types = np.unique(list(matched_synapses[segment_id].values()))
# label_matches = np.array(list(matched_synapses[segment_id].items())).T
# synapses_by_compartment = dict()
# synapse_centroids_by_compartment = dict()
# for synapse_label_type in synapse_label_types:
#     synapses_by_compartment[synapse_label_type] = label_matches[0][np.where(label_matches[1]==synapse_label_type)[0]]
#     centroids = np.zeros((len(synapses_by_compartment[synapse_label_type]), 3), dtype=np.uint32)
#     for i, specific_synapse in enumerate(synapses_by_compartment[synapse_label_type]):
#         centroids[i] = mesh_synapses[segment_id].T[1:].T[mesh_synapses[segment_id].T[0]==specific_synapse]
#     synapse_centroids_by_compartment[synapse_label_type] = centroids
# p3.figure(width=1024, height=1024)
# vertices, triangles = meshes[segment_id]
# p3.plot_trisurf(*vertices.T/1000, triangles=triangles, color='black')
# for label_type, label_name, label_color in label_keys:
#     if label_type in synapse_label_types:
#         print("{} -> {} : {}".format(label_type, label_name, label_color))
#         p3.scatter(*(synapse_centroids_by_compartment[label_type] * [4, 4, 40]).T/1000, color=label_color, marker='sphere', size=0.1)
# p3.squarelim()
# p3.show()